# Lab 1: Building your Agent 

The agent is comprised of a router using OpenAI function calling, and a set of three tools: a database lookup tool, a data analysis tool, and a code generator to create graphs.

<img src="images/agent.png" width="500"/>


The agent can lookup information from a local file, perform analysis on that information, and graph results. The example local file is a log of transactions at a local store. The agent can help the store owners understand trends and anomalies in their sales data.

## Importing necessary libraries 

In [45]:
from openai import OpenAI
import pandas as pd
import json
import duckdb
from pydantic import BaseModel, Field
from IPython.display import Markdown

from helper import get_openai_api_key
from helper import get_groq_api_key

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Your results might differ from those shown in the video.</p>

## Initializing the OpenAI client

In [46]:
# initialize the Groq client
# groq_api_key = get_groq_api_key()
client = OpenAI(api_key=get_openai_api_key)
# from groq import Groq

# client = Groq(api_key=groq_api_key)
MODEL = "gpt-4o-mini"
# MODEL = "openai/gpt-oss-120b"

## Defining the tools

Let's start by creating the three tools the agent will be able to use.

### Tool 1: Database Lookup

This first tool reads from a local parquet file that contains the transaction data. 

In [47]:
# define the path to the transactional data
# TRANSACTION_DATA_FILE_PATH = 'data/Store_Sales_Price_Elasticity_Promotions_Data.parquet'
DATASET_FILE_PATH = 'data/Titanic-Dataset.csv'

This database lookup tool works using three steps. 

<img src="images/tool1.png" width="500"/>

1. First, it creates the SQL table from a local file, if not already done.
2. Second, it translates the original prompt into an sql query (using an LLM call).
3. Finally, it runs that query against the database.

In [48]:
# prompt template for step 2 of tool 1
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

In [49]:
# code for step 2 of tool 1
def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    """Generate an SQL query based on a prompt"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt, 
                                                    columns=columns, 
                                                    table_name=table_name)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    return response.choices[0].message.content

In [50]:
# code for tool 1
def lookup_given_dataset(prompt: str) -> str:
    """Implementation of given dataset csv file using SQL"""
    try:

        # define the table name
        table_name = "dataset"
        
        # step 1: read the parquet file into a DuckDB table
        # df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
        df = pd.read_csv(DATASET_FILE_PATH)
        duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

        # step 2: generate the SQL code
        sql_query = generate_sql_query(prompt, df.columns, table_name)
        # clean the response to make sure it only includes the SQL code
        sql_query = sql_query.strip()
        sql_query = sql_query.replace("```sql", "").replace("```", "")
        
        # step 3: execute the SQL query
        result = duckdb.sql(sql_query).df()
        
        return result.to_string()
    except Exception as e:
        return f"Error accessing data: {str(e)}"

Great! Now let's test the first tool and make sure it worked correctly.

In [51]:
example_data = lookup_given_dataset("just return random 20 records from the titanic dataset")
print(example_data)

    PassengerId  Survived  Pclass                                                       Name     Sex    Age  SibSp  Parch           Ticket     Fare    Cabin Embarked
0           816         0       1                                           Fry, Mr. Richard    male    NaN      0      0           112058   0.0000     B102        S
1           861         0       3                                    Hansen, Mr. Claus Peter    male  41.00      2      0           350026  14.1083     None        S
2           525         0       3                                          Kassem, Mr. Fared    male    NaN      0      0             2700   7.2292     None        C
3           311         1       1                             Hays, Miss. Margaret Bechstein  female  24.00      0      0            11767  83.1583      C54        C
4           695         0       1                                            Weir, Col. John    male  60.00      0      0           113800  26.5500     None        S
5   

### Tool 2: Data Analysis

The second tool can analyze the returned data and display conclusions to users.

<img src="images/tool2.png" width="300"/>

In [52]:
# Construct prompt based on analysis type and data subset
DATA_ANALYSIS_PROMPT = """
Analyze the following data: {data}
Your job is to answer the following question: {prompt}
"""

In [53]:
# code for tool 2
def analyze_given_dataset(prompt: str, data: str) -> str:
    """Implementation of AI-powered Titanic data analysis"""
    formatted_prompt = DATA_ANALYSIS_PROMPT.format(data=data, prompt=prompt)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    analysis = response.choices[0].message.content
    return analysis if analysis else "No analysis could be generated"

This tool is relatively simple, but let's still test it out to be sure things are working correctly.

In [54]:
Markdown(analyze_given_dataset(prompt="what trends do you see in this data", 
                         data=example_data))

Analyzing the provided dataset related to the survival of Titanic passengers, we can observe several trends and patterns based on various attributes. Here are some key points:

1. **Survival Rates by Gender**:
   - Among the entries, there are both male and female passengers. The survival rates appear to differ by sex, aligning with historical records that indicate women and children had a higher chance of survival during the Titanic disaster. For instance, both entries for Miss. Margaret Bechstein Hays and Miss. Hilda Mary Slayter survived, while many male passengers did not.

2. **Pclass Influence**:
   - The dataset includes passengers from different classes (1, 2, 3). Higher-class passengers (Pclass 1) seem to have a higher chance of survival. For example, the only two passengers who survived from Pclass 1 in the provided data are both female. This is supported by historical statistics showing that first-class passengers had better access to lifeboats.

3. **Age Factor**:
   - The ages of passengers vary significantly. There are both very young (e.g., Master. Arthur Rice at 4 years old) and older passengers (e.g., Col. John Weir at 60). Survival rates seem to favor younger individuals and particularly females. The youngest survivor noted is the infant Caldwell, highlighting the tendency to prioritize women's and children's safety during the evacuation.

4. **Family Relations**:
   - The "SibSp" (siblings/spouses) and "Parch" (parents/children) columns indicate family connections. Some survivors like Caldwell (Master. Alden Gates) had family members aboard, while many passengers without family onboard did not survive. This suggests that family units may have been prioritized during rescues or had opportunities to assist each other in survival.

5. **Ticket Fare & Class**:
   - There is a strong connection between the ticket fare and the passenger class. Higher fares typically correspond to higher classes. Those who paid a more considerable fare (often in Pclass 1) stand a better chance of survival, underscoring historical patterns of social class impact on survival rates.

6. **Embarkation Points**:
   - The "Embarked" column shows where each passenger boarded (C, Q, S). While more extensive data would be needed for a robust analysis, this may provide insights about survival patterns based on embarkation location. 

7. **Missing Ages**:
   - Several records have missing age values (NaN), which could skew analysis or be indicative of incomplete data records. Understanding how missing data correlates with survival could be vital for precise insights.

8. **Cabin Values**:
   - The presence or absence of cabin data might indicate social status. Those with cabin assignments are more likely to belong to a higher class and tend to have higher survival rates. This characteristic could be analyzed further in relation to how the placement in the ship correlates with survival outcomes.

Overall, these trends highlight significant correlations between sex, class, age, family relations, and the likelihood of survival in the Titanic disaster. A deeper statistical analysis and larger dataset would allow for more robust conclusions.

### Tool 3: Data Visualization

The third tool generates python code to create the requested graphs from the returned data of the first tool. It consists of two steps:
<img src="images/tool3.png" width="500"/>
1. First, it creates the chart configuration: chart type, title, data, lables for x-axis and y-axis (using an LLM call).
2. Second, it generates the python code based on the chart configuration of the first step (using an LLM call).

In [55]:
# prompt template for step 1 of tool 3
CHART_CONFIGURATION_PROMPT = """
Generate a chart configuration based on this data: {data}
The goal is to show: {visualization_goal}
"""

In [56]:
# class defining the response format of step 1 of tool 3
class VisualizationConfig(BaseModel):
    chart_type: str = Field(..., description="Type of chart to generate")
    x_axis: str = Field(..., description="Name of the x-axis column")
    y_axis: str = Field(..., description="Name of the y-axis column")
    title: str = Field(..., description="Title of the chart")

In [57]:
# code for step 1 of tool 3
def extract_chart_config(data: str, visualization_goal: str) -> dict:
    """Generate chart visualization configuration
    
    Args:
        data: String containing the data to visualize
        visualization_goal: Description of what the visualization should show
        
    Returns:
        Dictionary containing user required chart configuration
    """
    formatted_prompt = CHART_CONFIGURATION_PROMPT.format(data=data,
                                                         visualization_goal=visualization_goal)

    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
        response_format=VisualizationConfig,
    )
    
    try:
        # Extract axis and title info from response
        content = response.choices[0].message.content
        
        # Return structured chart config
        return {
            "chart_type": content.chart_type,
            "x_axis": content.x_axis,
            "y_axis": content.y_axis,
            "title": content.title,
            "data": data
        }
    except Exception:
        return {
            "chart_type": "line", 
            "x_axis": "date",
            "y_axis": "value",
            "title": visualization_goal,
            "data": data
        }

In [58]:
# prompt template for step 2 of tool 3
CREATE_CHART_PROMPT = """
Write python code to create a chart based on the following configuration.
Only return the code, no other text.
config: {config}
"""

In [59]:
# code for step 2 of tool 3
def create_chart(config: dict) -> str:
    """Create a chart based on the configuration"""
    formatted_prompt = CREATE_CHART_PROMPT.format(config=config)
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    code = response.choices[0].message.content
    code = code.replace("```python", "").replace("```", "")
    code = code.strip()
    
    return code

In [60]:
# code for tool 3
def generate_visualization(data: str, visualization_goal: str) -> str:
    """Generate a visualization based on the data and goal"""
    config = extract_chart_config(data, visualization_goal)
    code = create_chart(config)
    try:
        exec(code)
    except Exception as e:
        print(f"Error executing visualization code: {str(e)}")
    return code

Great, now let's try the third tool out:

In [61]:
code = generate_visualization(example_data, 
                              "Show a bar chart of the number of survivors by age")

Error executing visualization code: Error tokenizing data. C error: Expected 15 fields in line 3, saw 16



<string>:29: SyntaxWarning: invalid escape sequence '\s'


## Defining the Router

Now that all of the tools are defined, you can create the router. The router will take the original user input, and is responsible for calling any tools. After each tool call is completed, the agent will return to router to determine whether another tool should be called.

### Tool Schema

Let's define the tools in a way that can be understood by our OpenAI model. OpenAI understands a specific JSON format:

In [62]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "lookup_given_dataset",
            "description": "Look up data from a given dataset",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_given_dataset", 
            "description": "Analyze given data to extract insights",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "The lookup_given_dataset tool's output."},
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["data", "prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_visualization",
            "description": "Generate Python matplotlib code to create data visualizations",
            "parameters": {
                "type": "object", 
                "properties": {
                    "data": {"type": "string", "description": "The lookup_given_dataset tool's output."},
                    "visualization_goal": {"type": "string", "description": "The goal of the visualization."}
                },
                "required": ["data", "visualization_goal"]
            }
        }
    }
]

# Dictionary mapping function names to their implementations
tool_implementations = {
    "lookup_given_dataset": lookup_given_dataset,
    "analyze_given_dataset": analyze_given_dataset, 
    "generate_visualization": generate_visualization
}

### Router Logic

The router is composed of a main loop method, and a method to handle the tool calls that you get back from the model.

<img src="images/router.png" width="800"/>

The following two cells define the function `handle_tool_calls` and the variable `SYSTEM_PROMPT`, which will be used by the function `run_agent` defining the router logic.

In [63]:
# code for executing the tools returned in the model's response
def handle_tool_calls(tool_calls, messages):
    
    for tool_call in tool_calls:   
        function = tool_implementations[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [64]:
SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions about the user given dataset.
"""

In [65]:
def run_agent(messages):
    print("Running agent with messages:", messages)

    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)

    while True:
        print("Making router call to OpenAI")
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
        )
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        print("Received response with tool calls:", bool(tool_calls))

        # if the model decides to call function(s), call handle_tool_calls
        if tool_calls:
            print("Processing tool calls")
            messages = handle_tool_calls(tool_calls, messages)
        else:
            print("No tool calls, returning final response")
            return response.choices[0].message.content

In [68]:
result = run_agent('who are you?')

Running agent with messages: who are you?
Making router call to OpenAI
Received response with tool calls: False
No tool calls, returning final response


In [69]:
# print(result)
# you can also print a formatted version of the result
Markdown(result)

I'm your helpful assistant, here to assist you with questions and insights related to your dataset. How can I help you today?

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>